In [18]:
import gspread
from google.colab import auth
from google.auth import default
from ipywidgets import widgets, HBox, VBox
from IPython.display import display

# 1. Setup & Authentication
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/example_spreedshrrt_link/edit?usp=sharing"
sheet = gc.open_by_url(spreadsheet_url).sheet1

# 2. Get Date Once
today_date = input("Enter today's date (e.g., 01/15): ")
data = sheet.get_all_values()
headers = data[0]
roll_list = [row[0] for row in data]

# Find or Create Column
if today_date in headers:
    col_idx = headers.index(today_date) + 1
else:
    col_idx = len(headers) + 1
    sheet.update_cell(1, col_idx, today_date)

# 3. Create UI for 3 Branches
cse_input = widgets.Text(description='CSE (001...):', placeholder='Space separated')
dsai_input = widgets.Text(description='DSAI:', placeholder='Space separated')
ece_input = widgets.Text(description='ECE:', placeholder='Space separated')
submit_btn = widgets.Button(description="Submit All Attendance", button_style='success')
output = widgets.Output()

def process_attendance(b):
    with output:
        output.clear_output()
        branch_map = {"23BCS": cse_input.value, "23BDS": dsai_input.value, "23BEC": ece_input.value}
        updates = []

        for prefix, values in branch_map.items():
            if not values.strip(): continue
            for r in values.split():
                full_roll = f"{prefix}{str(r).zfill(3)}"
                if full_roll in roll_list:
                    row_idx = roll_list.index(full_roll) + 1
                    # Batch check and update logic
                    if sheet.cell(row_idx, col_idx).value == '1':
                        print(f"⚠️ {full_roll} already present.")
                    else:
                        updates.append({'range': gspread.utils.rowcol_to_a1(row_idx, col_idx), 'values': [[1]]})
                else:
                    print(f"❌ {full_roll} not found.")

        if updates:
            sheet.batch_update(updates)
            print(f"✅ Successfully marked {len(updates)} new students.")

submit_btn.on_click(process_attendance)
display(VBox([cse_input, dsai_input, ece_input, submit_btn, output]))

Enter today's date (e.g., 01/15): 01/17
